dataset_fl

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import os
import random
import json

# --- 장치 설정 (GPU 사용 가능 시) ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# -------------------- 데이터 로딩 및 전처리 --------------------
# 'dataset_fl.json' 파일을 읽어옵니다.
try:
    df = pd.read_json('dataset_fl.json', lines=True)
except ValueError:
    print("Trying to read JSON without lines=True (assuming a single JSON object or array of objects).")
    df = pd.read_json('dataset_fl.json')

df_processed = df[['user_id', 'business_id', 'review_stars']].copy()

# Label Encoding을 사용하여 user_id와 business_id를 정수형으로 변환합니다.
user_encoder = LabelEncoder()
business_encoder = LabelEncoder()
df_processed['user_encoded'] = user_encoder.fit_transform(df_processed['user_id'])
df_processed['business_encoded'] = business_encoder.fit_transform(df_processed['business_id'])

num_users = len(user_encoder.classes_)
num_businesses = len(business_encoder.classes_)
print(f"✅ 데이터 로딩 및 전처리 완료. 사용자 수: {num_users}, 비즈니스 수: {num_businesses}")

# -------------------- Dataset 정의 --------------------
class NeuMFDataset(Dataset):
    def __init__(self, df):
        self.user_ids = torch.tensor(df['user_encoded'].values, dtype=torch.long)
        self.item_ids = torch.tensor(df['business_encoded'].values, dtype=torch.long)
        self.ratings = torch.tensor(df['review_stars'].values, dtype=torch.float)

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        return self.user_ids[idx], self.item_ids[idx], self.ratings[idx]

# -------------------- 모델 정의 --------------------
class NeuMF(nn.Module):
    def __init__(self, num_users, num_items, mf_dim, mlp_dims):
        super(NeuMF, self).__init__()
        self.user_embedding_gmf = nn.Embedding(num_users, mf_dim)
        self.item_embedding_gmf = nn.Embedding(num_items, mf_dim)
        
        self.user_embedding_mlp = nn.Embedding(num_users, mlp_dims[0] // 2)
        self.item_embedding_mlp = nn.Embedding(num_items, mlp_dims[0] // 2)

        mlp_layers = []
        input_dim = mlp_dims[0]
        for dim in mlp_dims[1:]:
            mlp_layers.append(nn.Linear(input_dim, dim))
            mlp_layers.append(nn.ReLU())
            input_dim = dim
        self.mlp = nn.Sequential(*mlp_layers)

        self.final_layer = nn.Linear(mf_dim + mlp_dims[-1], 1)

    def forward(self, user_ids, item_ids):
        gmf_user = self.user_embedding_gmf(user_ids)
        gmf_item = self.item_embedding_gmf(item_ids)
        gmf_output = gmf_user * gmf_item

        mlp_user = self.user_embedding_mlp(user_ids)
        mlp_item = self.item_embedding_mlp(item_ids)
        mlp_input = torch.cat((mlp_user, mlp_item), dim=1)
        mlp_output = self.mlp(mlp_input)

        concat = torch.cat((gmf_output, mlp_output), dim=1)
        prediction = self.final_layer(concat)
        return prediction.view(-1)

# -------------------- 평가 지표 함수 정의 --------------------
def evaluate_model(model, data_loader, device):
    model.eval()
    preds, targets = [], []
    with torch.no_grad():
        for user_ids, item_ids, ratings in data_loader:
            user_ids, item_ids, ratings = user_ids.to(device), item_ids.to(device), ratings.to(device)
            output = model(user_ids, item_ids)
            preds.extend(output.cpu().numpy())
            targets.extend(ratings.cpu().numpy())
    
    preds = np.array(preds)
    targets = np.array(targets)
    
    mae = mean_absolute_error(targets, preds)
    rmse = np.sqrt(mean_squared_error(targets, preds))
    
    return mae, rmse

# -------------------- 학습 및 평가 함수 (단일 실행) --------------------
def train_and_evaluate_run(params, train_df, val_df, test_df, run_num):
    print(f"--- 실행 {run_num} 시작, 파라미터: {params}")

    train_dataset = NeuMFDataset(train_df)
    val_dataset = NeuMFDataset(val_df)
    test_dataset = NeuMFDataset(test_df)

    train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=params['batch_size'], shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=params['batch_size'], shuffle=False)
    
    model = NeuMF(num_users, num_businesses, mf_dim=params['mf_dim'], mlp_dims=params['mlp_dims']).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])
    model_path = f'best_model_run_{run_num}.pt'

    best_val_rmse = float('inf')
    epochs_no_improve = 0
    patience = 5
    min_delta = 0.0001
    epochs = 50

    for epoch in range(epochs):
        model.train()
        for user_ids, item_ids, ratings in train_loader:
            user_ids, item_ids, ratings = user_ids.to(device), item_ids.to(device), ratings.to(device)
            optimizer.zero_grad()
            predictions = model(user_ids, item_ids)
            loss = criterion(predictions, ratings)
            loss.backward()
            optimizer.step()

        val_mae, val_rmse = evaluate_model(model, val_loader, device)

        if val_rmse < best_val_rmse - min_delta:
            best_val_rmse = val_rmse
            epochs_no_improve = 0
            torch.save(model.state_dict(), model_path)
        else:
            epochs_no_improve += 1
            if epochs_no_improve == patience:
                print(f"조기 종료 발생. (실행 {run_num})")
                break
    
    if os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path))
    else:
        print(f"최적 모델을 찾지 못해 현재 모델을 사용합니다. (실행 {run_num})")

    test_mae, test_rmse = evaluate_model(model, test_loader, device)
    
    if os.path.exists(model_path):
        os.remove(model_path)

    return test_mae, test_rmse, best_val_rmse

# -------------------- 메인 실행 루틴 --------------------
def main():
    # -------------------- 1. 하이퍼파라미터 랜덤 서치 --------------------
    param_grid = {
        'mf_dim': [16, 32, 64],
        'mlp_dims': [[64, 32, 16], [128, 64, 32], [256, 128, 64]],
        'learning_rate': [0.0005, 0.001, 0.002],
        'batch_size': [128, 256, 512]
    }
    num_trials = 10
    best_params = None
    best_val_rmse_search = float('inf')

    print(f"\n--- 1단계: 하이퍼파라미터 랜덤 서치 ({num_trials}회) 시작 ---")
    
    # 랜덤 서치를 위한 데이터 분할 (테스트 셋을 따로 떼어 놓음)
    train_val_df_search, test_df_for_final = train_test_split(df_processed, test_size=0.2, random_state=42)
    train_df_search, val_df_search = train_test_split(train_val_df_search, test_size=1/8, random_state=42)

    for i in range(num_trials):
        current_params = {k: random.choice(v) for k, v in param_grid.items()}
        print(f"\n--- 시도 {i+1}/{num_trials} ---")
        
        # 임시 실행으로 최적의 검증 RMSE를 찾음
        _, _, current_val_rmse = train_and_evaluate_run(current_params, train_df_search, val_df_search, test_df_for_final, f'search_{i+1}')
        
        print(f"    --> 시도 {i+1} 검증 RMSE: {current_val_rmse:.4f}")
        
        if current_val_rmse < best_val_rmse_search:
            best_val_rmse_search = current_val_rmse
            best_params = current_params
            print(f"    --> 새로운 최적 RMSE 발견: {best_val_rmse_search:.4f} (파라미터: {best_params})")

    print(f"\n✅ 1단계 완료. 최적 파라미터: {best_params}")

    # -------------------- 2. 최적 파라미터로 5회 반복 최종 평가 --------------------
    if best_params:
        all_rmse, all_mae = [], []
        num_runs = 5
        print(f"\n--- 2단계: 최적 파라미터로 {num_runs}회 반복 평가 시작 ---")

        for i in range(num_runs):
            # 매번 새로운 무작위 데이터 분할 사용
            random_state = 42 + i
            train_val_df_final, test_df_final = train_test_split(df_processed, test_size=0.2, random_state=random_state)
            train_df_final, val_df_final = train_test_split(train_val_df_final, test_size=1/8, random_state=random_state)

            test_mae, test_rmse, _ = train_and_evaluate_run(best_params, train_df_final, val_df_final, test_df_final, f'final_{i+1}')
            
            print(f"\n✅ [NeuMF] {i+1}번째 테스트 결과: RMSE={test_rmse:.4f}, MAE={test_mae:.4f}")
            
            all_rmse.append(test_rmse)
            all_mae.append(test_mae)

        # 최종 평균 계산 및 출력
        avg_rmse = np.mean(all_rmse)
        std_rmse = np.std(all_rmse)
        avg_mae = np.mean(all_mae)
        std_mae = np.std(all_mae)

        print("\n\n==================== 최종 5회 반복 평균 결과 ====================")
        print(f" - 평균 RMSE : {avg_rmse:.4f} +/- {std_rmse:.4f}")
        print(f" - 평균 MAE  : {avg_mae:.4f} +/- {std_mae:.4f}")

if __name__ == "__main__":
    main()

Using device: cuda
✅ 데이터 로딩 및 전처리 완료. 사용자 수: 33779, 비즈니스 수: 9422

--- 1단계: 하이퍼파라미터 랜덤 서치 (10회) 시작 ---

--- 시도 1/10 ---
--- 실행 search_1 시작, 파라미터: {'mf_dim': 64, 'mlp_dims': [256, 128, 64], 'learning_rate': 0.001, 'batch_size': 512}
조기 종료 발생. (실행 search_1)
    --> 시도 1 검증 RMSE: 1.1971
    --> 새로운 최적 RMSE 발견: 1.1971 (파라미터: {'mf_dim': 64, 'mlp_dims': [256, 128, 64], 'learning_rate': 0.001, 'batch_size': 512})

--- 시도 2/10 ---
--- 실행 search_2 시작, 파라미터: {'mf_dim': 16, 'mlp_dims': [256, 128, 64], 'learning_rate': 0.0005, 'batch_size': 512}
조기 종료 발생. (실행 search_2)
    --> 시도 2 검증 RMSE: 1.1982

--- 시도 3/10 ---
--- 실행 search_3 시작, 파라미터: {'mf_dim': 32, 'mlp_dims': [128, 64, 32], 'learning_rate': 0.001, 'batch_size': 512}
조기 종료 발생. (실행 search_3)
    --> 시도 3 검증 RMSE: 1.1965
    --> 새로운 최적 RMSE 발견: 1.1965 (파라미터: {'mf_dim': 32, 'mlp_dims': [128, 64, 32], 'learning_rate': 0.001, 'batch_size': 512})

--- 시도 4/10 ---
--- 실행 search_4 시작, 파라미터: {'mf_dim': 32, 'mlp_dims': [64, 32, 16], 'learning_rate': 0.

dataset_la

dataset_la

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import os
import random
import json

# --- 장치 설정 (GPU 사용 가능 시) ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# -------------------- 데이터 로딩 및 전처리 --------------------
# 'dataset_fl.json' 파일을 읽어옵니다.
try:
    df = pd.read_json('dataset_la.json', lines=True)
except ValueError:
    print("Trying to read JSON without lines=True (assuming a single JSON object or array of objects).")
    df = pd.read_json('dataset_la.json')

df_processed = df[['user_id', 'business_id', 'review_stars']].copy()

# Label Encoding을 사용하여 user_id와 business_id를 정수형으로 변환합니다.
user_encoder = LabelEncoder()
business_encoder = LabelEncoder()
df_processed['user_encoded'] = user_encoder.fit_transform(df_processed['user_id'])
df_processed['business_encoded'] = business_encoder.fit_transform(df_processed['business_id'])

num_users = len(user_encoder.classes_)
num_businesses = len(business_encoder.classes_)
print(f"✅ 데이터 로딩 및 전처리 완료. 사용자 수: {num_users}, 비즈니스 수: {num_businesses}")

# -------------------- Dataset 정의 --------------------
class NeuMFDataset(Dataset):
    def __init__(self, df):
        self.user_ids = torch.tensor(df['user_encoded'].values, dtype=torch.long)
        self.item_ids = torch.tensor(df['business_encoded'].values, dtype=torch.long)
        self.ratings = torch.tensor(df['review_stars'].values, dtype=torch.float)

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        return self.user_ids[idx], self.item_ids[idx], self.ratings[idx]

# -------------------- 모델 정의 --------------------
class NeuMF(nn.Module):
    def __init__(self, num_users, num_items, mf_dim, mlp_dims):
        super(NeuMF, self).__init__()
        self.user_embedding_gmf = nn.Embedding(num_users, mf_dim)
        self.item_embedding_gmf = nn.Embedding(num_items, mf_dim)
        
        self.user_embedding_mlp = nn.Embedding(num_users, mlp_dims[0] // 2)
        self.item_embedding_mlp = nn.Embedding(num_items, mlp_dims[0] // 2)

        mlp_layers = []
        input_dim = mlp_dims[0]
        for dim in mlp_dims[1:]:
            mlp_layers.append(nn.Linear(input_dim, dim))
            mlp_layers.append(nn.ReLU())
            input_dim = dim
        self.mlp = nn.Sequential(*mlp_layers)

        self.final_layer = nn.Linear(mf_dim + mlp_dims[-1], 1)

    def forward(self, user_ids, item_ids):
        gmf_user = self.user_embedding_gmf(user_ids)
        gmf_item = self.item_embedding_gmf(item_ids)
        gmf_output = gmf_user * gmf_item

        mlp_user = self.user_embedding_mlp(user_ids)
        mlp_item = self.item_embedding_mlp(item_ids)
        mlp_input = torch.cat((mlp_user, mlp_item), dim=1)
        mlp_output = self.mlp(mlp_input)

        concat = torch.cat((gmf_output, mlp_output), dim=1)
        prediction = self.final_layer(concat)
        return prediction.view(-1)

# -------------------- 평가 지표 함수 정의 --------------------
def evaluate_model(model, data_loader, device):
    model.eval()
    preds, targets = [], []
    with torch.no_grad():
        for user_ids, item_ids, ratings in data_loader:
            user_ids, item_ids, ratings = user_ids.to(device), item_ids.to(device), ratings.to(device)
            output = model(user_ids, item_ids)
            preds.extend(output.cpu().numpy())
            targets.extend(ratings.cpu().numpy())
    
    preds = np.array(preds)
    targets = np.array(targets)
    
    mae = mean_absolute_error(targets, preds)
    rmse = np.sqrt(mean_squared_error(targets, preds))
    
    return mae, rmse

# -------------------- 학습 및 평가 함수 (단일 실행) --------------------
def train_and_evaluate_run(params, train_df, val_df, test_df, run_num):
    print(f"--- 실행 {run_num} 시작, 파라미터: {params}")

    train_dataset = NeuMFDataset(train_df)
    val_dataset = NeuMFDataset(val_df)
    test_dataset = NeuMFDataset(test_df)

    train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=params['batch_size'], shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=params['batch_size'], shuffle=False)
    
    model = NeuMF(num_users, num_businesses, mf_dim=params['mf_dim'], mlp_dims=params['mlp_dims']).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])
    model_path = f'best_model_run_{run_num}.pt'

    best_val_rmse = float('inf')
    epochs_no_improve = 0
    patience = 5
    min_delta = 0.0001
    epochs = 50

    for epoch in range(epochs):
        model.train()
        for user_ids, item_ids, ratings in train_loader:
            user_ids, item_ids, ratings = user_ids.to(device), item_ids.to(device), ratings.to(device)
            optimizer.zero_grad()
            predictions = model(user_ids, item_ids)
            loss = criterion(predictions, ratings)
            loss.backward()
            optimizer.step()

        val_mae, val_rmse = evaluate_model(model, val_loader, device)

        if val_rmse < best_val_rmse - min_delta:
            best_val_rmse = val_rmse
            epochs_no_improve = 0
            torch.save(model.state_dict(), model_path)
        else:
            epochs_no_improve += 1
            if epochs_no_improve == patience:
                print(f"조기 종료 발생. (실행 {run_num})")
                break
    
    if os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path))
    else:
        print(f"최적 모델을 찾지 못해 현재 모델을 사용합니다. (실행 {run_num})")

    test_mae, test_rmse = evaluate_model(model, test_loader, device)
    
    if os.path.exists(model_path):
        os.remove(model_path)

    return test_mae, test_rmse, best_val_rmse

# -------------------- 메인 실행 루틴 --------------------
def main():
    # -------------------- 1. 하이퍼파라미터 랜덤 서치 --------------------
    param_grid = {
        'mf_dim': [16, 32, 64],
        'mlp_dims': [[64, 32, 16], [128, 64, 32], [256, 128, 64]],
        'learning_rate': [0.0005, 0.001, 0.002],
        'batch_size': [128, 256, 512]
    }
    num_trials = 10
    best_params = None
    best_val_rmse_search = float('inf')

    print(f"\n--- 1단계: 하이퍼파라미터 랜덤 서치 ({num_trials}회) 시작 ---")
    
    # 랜덤 서치를 위한 데이터 분할 (테스트 셋을 따로 떼어 놓음)
    train_val_df_search, test_df_for_final = train_test_split(df_processed, test_size=0.2, random_state=42)
    train_df_search, val_df_search = train_test_split(train_val_df_search, test_size=1/8, random_state=42)

    for i in range(num_trials):
        current_params = {k: random.choice(v) for k, v in param_grid.items()}
        print(f"\n--- 시도 {i+1}/{num_trials} ---")
        
        # 임시 실행으로 최적의 검증 RMSE를 찾음
        _, _, current_val_rmse = train_and_evaluate_run(current_params, train_df_search, val_df_search, test_df_for_final, f'search_{i+1}')
        
        print(f"    --> 시도 {i+1} 검증 RMSE: {current_val_rmse:.4f}")
        
        if current_val_rmse < best_val_rmse_search:
            best_val_rmse_search = current_val_rmse
            best_params = current_params
            print(f"    --> 새로운 최적 RMSE 발견: {best_val_rmse_search:.4f} (파라미터: {best_params})")

    print(f"\n✅ 1단계 완료. 최적 파라미터: {best_params}")

    # -------------------- 2. 최적 파라미터로 5회 반복 최종 평가 --------------------
    if best_params:
        all_rmse, all_mae = [], []
        num_runs = 5
        print(f"\n--- 2단계: 최적 파라미터로 {num_runs}회 반복 평가 시작 ---")

        for i in range(num_runs):
            # 매번 새로운 무작위 데이터 분할 사용
            random_state = 42 + i
            train_val_df_final, test_df_final = train_test_split(df_processed, test_size=0.2, random_state=random_state)
            train_df_final, val_df_final = train_test_split(train_val_df_final, test_size=1/8, random_state=random_state)

            test_mae, test_rmse, _ = train_and_evaluate_run(best_params, train_df_final, val_df_final, test_df_final, f'final_{i+1}')
            
            print(f"\n✅ [NeuMF] {i+1}번째 테스트 결과: RMSE={test_rmse:.4f}, MAE={test_mae:.4f}")
            
            all_rmse.append(test_rmse)
            all_mae.append(test_mae)

        # 최종 평균 계산 및 출력
        avg_rmse = np.mean(all_rmse)
        std_rmse = np.std(all_rmse)
        avg_mae = np.mean(all_mae)
        std_mae = np.std(all_mae)

        print("\n\n==================== 최종 5회 반복 평균 결과 ====================")
        print(f" - 평균 RMSE : {avg_rmse:.4f} +/- {std_rmse:.4f}")
        print(f" - 평균 MAE  : {avg_mae:.4f} +/- {std_mae:.4f}")

if __name__ == "__main__":
    main()

Using device: cuda
✅ 데이터 로딩 및 전처리 완료. 사용자 수: 23621, 비즈니스 수: 3897

--- 1단계: 하이퍼파라미터 랜덤 서치 (10회) 시작 ---

--- 시도 1/10 ---
--- 실행 search_1 시작, 파라미터: {'mf_dim': 16, 'mlp_dims': [64, 32, 16], 'learning_rate': 0.0005, 'batch_size': 256}
조기 종료 발생. (실행 search_1)
    --> 시도 1 검증 RMSE: 1.1374
    --> 새로운 최적 RMSE 발견: 1.1374 (파라미터: {'mf_dim': 16, 'mlp_dims': [64, 32, 16], 'learning_rate': 0.0005, 'batch_size': 256})

--- 시도 2/10 ---
--- 실행 search_2 시작, 파라미터: {'mf_dim': 16, 'mlp_dims': [128, 64, 32], 'learning_rate': 0.002, 'batch_size': 512}
조기 종료 발생. (실행 search_2)
    --> 시도 2 검증 RMSE: 1.1220
    --> 새로운 최적 RMSE 발견: 1.1220 (파라미터: {'mf_dim': 16, 'mlp_dims': [128, 64, 32], 'learning_rate': 0.002, 'batch_size': 512})

--- 시도 3/10 ---
--- 실행 search_3 시작, 파라미터: {'mf_dim': 64, 'mlp_dims': [256, 128, 64], 'learning_rate': 0.001, 'batch_size': 256}
조기 종료 발생. (실행 search_3)
    --> 시도 3 검증 RMSE: 1.1224

--- 시도 4/10 ---
--- 실행 search_4 시작, 파라미터: {'mf_dim': 32, 'mlp_dims': [128, 64, 32], 'learning_rate': 0.00